In [1]:
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

from models.transformer import create_vit_model, fit_vit_model
from models.utils.get_train_data import get_average_green_band_data
from models.utils.root_mean_squared_error import root_mean_squared_error
import os.path as osp
import csv
import data
import matplotlib.pyplot as plt
import models
import argparse
from models.segmenter import prepare_dataset
from models.utils.get_train_data import get_all_from_band

In [2]:
with open(osp.join(osp.dirname(data.__file__), 'patch_names'), newline='') as f:
    reader = csv.reader(f)
    patch_name_data = list(reader)
patch_names = patch_name_data[0]
# patch_names = patch_names[0:100]
train_data_path = osp.join(osp.dirname(data.__file__), "forest-biomass")

In [3]:
# patch_names = patch_names[0:100]

band_stds = []
band_means = []
for band in range(11):

    x = get_all_from_band(patch_names, band, train_data_path)
    print(x.shape)
    band_stds.append(np.std(x))
    band_stds.append(np.mean(x))


KeyboardInterrupt



In [4]:
sentinel_1_bands = {
    "VV ascending": 0,
    "VH ascending": 0,
    "VV descending": 0,
    "VH descending": 0
}

sentinel_2_bands = {
    "B2-Blue": 1,
    "B3-Green": 1,
    "B4-Red": 1,
    "B5-Veg red edge 1": 1,
    "B6-Veg red edge 2": 1,
    "B7-Veg red edge 3": 1,
    "B8-NIR": 1,
    "B8A-Narrow NIR": 1,
    "B11-SWIR 1": 1,
    "B12-SWIR 2": 1,
    "Cloud probability": 0
}

s1_list = list(sentinel_1_bands.values())
s2_list = list(sentinel_2_bands.values())

In [5]:
data_path = osp.dirname(data.__file__)

training_ids_path = osp.join(data_path, "patch_names")
training_features_path = osp.join(data_path, "forest-biomass")

train_dataset, number_of_channels = prepare_dataset(training_ids_path, training_features_path, s1_list, s2_list)

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=20)

In [9]:
from tqdm import tqdm
import torch


def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0

    for data, _ in tqdm(dataloader):
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1

    means = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    stds = (channels_squared_sum / num_batches - means ** 2) ** 0.5

    return means, stds

In [10]:
means, stds = get_mean_and_std(train_dataloader)

100%|██████████| 608/608 [00:15<00:00, 38.42it/s]


In [11]:
means

tensor([1666.0227, 1649.1184, 1641.2771, 1960.8578, 2520.4958, 2628.6143,
        2778.9536, 2723.2520, 1025.0494,  697.5798])

In [12]:
stds

tensor([2539.4539, 2355.2258, 2432.2163, 2428.7878, 2243.6016, 2131.9910,
        2225.4001, 2053.2605,  930.2753,  755.9792])

In [13]:
patch_names = patch_names[0:500]

band_stds = []
band_means = []
for band in range(10):

    x = get_all_from_band(patch_names, band, train_data_path)
    print(x.shape)
    band_stds.append(np.std(x))
    band_means.append(np.mean(x))

(4859, 256, 256)
(4859, 256, 256)
(4859, 256, 256)
(4859, 256, 256)
(4859, 256, 256)
(4859, 256, 256)
(4859, 256, 256)
(4859, 256, 256)
(4859, 256, 256)
(4859, 256, 256)


In [14]:
band_stds

[2540.3397510567966,
 2356.076757612205,
 2433.075508463788,
 2429.6509315323638,
 2244.3870278230097,
 2132.702299852526,
 2226.137935423653,
 2053.9244507257426,
 930.5311195966731,
 756.2597490490978]

In [15]:
band_means

[1667.4965786145876,
 1650.426576160997,
 1642.6580560954665,
 1962.2058812039554,
 2521.569034423326,
 2629.538568440571,
 2779.872157274094,
 2724.073150317594,
 1024.9350094949298,
 697.6740744387228]

In [ ]:
# Conclusion, normalizing was already being done in create_tensor